Import Packages

In [ ]:
from black_litterman_utils import *

# Input Parameters

Example Usage for the Client's Risk Level

In [ ]:
# Load information of asset class, proxy index, ticker, etc.
client_name = 'CCBA_demo'
client_risk_level = 'C3'  # C1 to C5
client_risk_preference = 'Standard'  # Defensive, Standard, Enhanced
market_caps, historical_returns = get_proxy_data(client_name, client_risk_level)
target_vol = risk_to_target_vol[client_risk_level][client_risk_preference]

Views for the Black-Litterman Model

In [ ]:
# Define views example
views = {
    # 绝对观点 (Absolute Views)
    # 格式: (资产名称, 年化预期收益率, 置信度)
    'absolute': [
        # 观点1: 亚太-中国 估值修复反弹
        # 逻辑: 政策刺激后的估值均值回归，波动大但赔率高
        ('亚太地区-中国', 0.15, 0.80),

        # 观点2: 能源板块 维持高位震荡及分红收益
        # 逻辑: 地缘政治供给受限 + 传统能源资本开支不足
        ('行业指数-能源', 0.11, 0.90),

        # 观点3: 长端美债 承受财政赤字压力
        # 逻辑: 尽管降息，但通胀粘性和美债供给过剩导致长端收益率难以下行，价格回报受限
        ('长端美国国债', -0.04, 0.95),
    ],
    # 相对观点 (Relative Views)
    # 格式: (做多资产, 做空资产, 差额收益率, 置信度)
    'relative': [
        # 观点4: 美股风格切换 (Size Factor Rotation)
        # 逻辑: 降息周期中段，利好高负债的小盘股；大型成长股面临高估值消化压力
        ('美股小型股', '美股大型成长股', 0.09, 0.80),

        # 观点5: 避险资产内部分化 (Real vs Nominal Assets)
        # 逻辑: 担心“再通胀”风险，实物黄金优于名义长债
        ('黄金', '长端美国国债', 0.16, 0.85),

        # 观点6: 新兴市场 vs 欧洲 (Growth Differential)
        # 逻辑: 亚洲新兴市场增长预期优于深陷滞胀风险的欧洲
        ('亚太地区（除日本）', '欧洲地区', 0.07, 0.70),

        # 观点7: 信用利差观点
        # 逻辑: 经济软着陆背景下，高收益债提供Carry，优于利率敏感的投资级债
        ('高收益公司债', '全球公司债', 0.065, 0.75),
    ]
}
# views = {'absolute': [('亚太地区-中国', 0.12, 0)]}
window = 2  # years
# Get BL mu and Sigma (using Ledoit-Wolf if desired: change method in compute_covariance)
mu_bl, Sigma_bl, w_eq_0 = black_litterman(historical_returns, market_caps, window, views)
# note: Rescale to match risk profile
w_eq = rescale_equilibrium_weights(w_eq_0, client_risk_level)
# output BL results
plot_matrix(Sigma_bl)
plot_series(mu_bl, series_type='Return')

In [ ]:
# Backtest Equilibrium Portfolio
print_weight_summary(w_eq, "Market-Equilibrium")
plot_series(w_eq, series_type='Weights')

nav_eq, stats_eq = backtest_portfolio(w_eq, historical_returns, window, rebalance=False)
print("\nMarket-Equilibrium Backtest Stats:")
for k, v in stats_eq.items():
    if k == 'Max Drawdown':
        print(f"  {k}: {v:.2%}")
    else:
        print(f"  {k}: {v:.2%}" if 'Return' in k or 'Volatility' in k else f"  {k}: {v:.2f}")
plot_backtest(nav_eq, title="Market-Equilibrium Portfolio Backtest")

# Model Portfolio Optimization

Mean-Variance

In [ ]:
parameters_mv = {
    'risk_level': client_risk_level,
    'expected_return': mu_bl,
    'covariance_matrix': Sigma_bl,
    'optimization_method': 'mean_variance',
    'target_vol': target_vol,
    'benchmark_weights': w_eq,  # ← crucial for broad class targets
}
weights_mv, vol_mv = get_portfolio_weight(parameters_mv)
print_weight_summary(weights_mv, "Mean-Variance")
plot_series(weights_mv, series_type='Weights')
print_internal_allocation(w_eq, weights_mv, "Mean-Variance")


In [ ]:
# Backtest MV
nav_mv, stats_mv = backtest_portfolio(weights_mv, historical_returns, window, rebalance=False)
print("\nMean-Variance Backtest Stats:")
for k, v in stats_mv.items():
    if k == 'Max Drawdown':
        print(f"  {k}: {v:.2%}")
    else:
        print(f"  {k}: {v:.2%}" if 'Return' in k or 'Volatility' in k else f"  {k}: {v:.2f}")
plot_backtest(nav_mv, title="Mean-Variance Portfolio Backtest")

In [ ]:
# comparison
nav_eq, nav_mv, stats_eq, stats_mv = compare_portfolios(
    w_eq, weights_mv,
    historical_returns,
    window=window,
    name_a="Market Equilibrium",
    name_b="Mean-Variance Optimized"
)

Risk-Parity

In [ ]:
parameters_rp = {
    'risk_level': client_risk_level,
    'expected_return': mu_bl,
    'covariance_matrix': Sigma_bl,
    'optimization_method': 'risk_parity',
}
weights_rp, vol_rp = get_portfolio_weight(parameters_rp)
print_weight_summary(weights_rp, "Risk Parity")
plot_series(weights_rp, series_type='Weights')


In [ ]:
# Backtest RP
nav_rp, stats_rp = backtest_portfolio(weights_rp, historical_returns, window, rebalance=False)
print("\nRisk Parity Backtest Stats:")
for k, v in stats_rp.items():
    if k == 'Max Drawdown':
        print(f"  {k}: {v:.2%}")
    else:
        print(f"  {k}: {v:.2%}" if 'Return' in k or 'Volatility' in k else f"  {k}: {v:.2f}")
plot_backtest(nav_rp, title="Risk Parity Portfolio Backtest")

In [ ]:
# comparison
nav_eq, nav_rp, stats_eq, stats_mv = compare_portfolios(
    w_eq, weights_rp,
    historical_returns,
    window=window,
    name_a="Market Equilibrium",
    name_b="Risk-Parity Optimized"
)

# Rolling Rebalance Backtest

In [ ]:
# Rolling backtest
optimization_method = 'mean_variance'
frequency = 'Y'
nav_rolling, port_hist, stats_rolling = rolling_rebalance_backtest(
    returns_df=historical_returns,
    market_caps=market_caps,
    views=views,
    period_years=5,
    rebalance_freq=frequency,  # yearly
    optimization_method=optimization_method,
    target_vol=target_vol,
    window=window,
    risk_level=client_risk_level,
)


In [ ]:
print("\n=== Rolling Rebalance Backtest Results ===")
for k, v in stats_rolling.items():
    if k == 'Max Drawdown':
        print(f"  {k}: {v:.2%}")
    elif 'Ratio' in k:
        print(f"  {k}: {v:.2f}")
    else:
        print(f"  {k}: {v:.2%}")

# Plot NAV
plot_backtest(nav_rolling, "Rolling Rebalance NAV")


In [ ]:
# Print and plot portfolio history
print_portfolio_history(port_hist, top_n=10)
plot_portfolio_history(port_hist, f"Portfolio Weight Evolution (Method={optimization_method}, Reb Frequency={frequency})")


# View sensitivity with performance

In [ ]:
baseline_w, sens_results = analyze_view_sensitivity(
    returns_df=historical_returns,
    market_caps=market_caps,
    full_views=views,
    window=window,
    optimization_method='mean_variance',
    risk_level=client_risk_level,
    target_vol=target_vol,
)

# Backtest baseline portfolio
nav_baseline, stats_baseline = backtest_portfolio(
    baseline_w, historical_returns, window=window, rebalance=False
)
print_view_sensitivity_results(baseline_w, sens_results, stats_baseline)

plot_view_sensitivity(sens_results, top_k=3)

In [ ]:
# plot NAVs for top view
if sens_results:
    compare_portfolios(
        baseline_w,
        sens_results[0]['weights'],
        historical_returns,
        window=window,
        name_a="With All Views",
        name_b=f"Without: {sens_results[0]['view_desc'][:30]}..."
    )